In [1]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator, ParameterEstimator, BicScore, ConstraintBasedEstimator
from pgmpy.inference import VariableElimination
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image

In [2]:

'''
Attribute Information:
   -- Only 14 used
      -- 1. #3  (age), age in years
      -- 2. #4  (sex), sex (1 = male; 0 = female)
      -- 3. #9  (cp), chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
      -- 4. #10 (trestbps), resting blood pressure (in mm Hg on admission to the hospital)
      -- 5. #12 (chol), serum cholestoral in mg/dl
      -- 6. #16 (fbs), (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
      -- 7. #19 (restecg), restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
      -- 8. #32 (thalach), maximum heart rate achieved
      -- 9. #38 (exang), exercise induced angina (1 = yes; 0 = no)
      -- 10. #40 (oldpeak), = ST depression induced by exercise relative to rest
      -- 11. #41 (slope), the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping   
      -- 12. #44 (ca), number of major vessels (0-3) colored by flourosopy
      -- 13. #51 (thal), 3 = normal; 6 = fixed defect; 7 = reversable defect 
      -- 14. #58 (num), (the predicted attribute) num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)



10. Class Distribution:
        Database:      0   1   2   3   4 Total
          Cleveland: 164  55  36  35  13   303
          Hungarian: 188  37  26  28  15   294
        Switzerland:   8  48  32  30   5   123
      Long Beach VA:  51  56  41  42  10   200


TODO: * Check if there is a -9 in the dataset because it is stated that this would be the marker for a missing value.
      * Check for occurences of ?
'''



column_names = [
    'age',
    'sex',
    'cp',
    'trestbps',
    'chol',
    'fbs',
    'restecg',
    'thalach',
    'exang',
    'oldpeak',
    'slope',
    'ca',
    'thal', 
    'num'
]

df = pd.read_csv("data/processed.cleveland.data", header = None, names = column_names)

# Removing non-numeric values
for column in column_names:
    # pandas.to_numeric return type depends on input. Series if Series, otherwise ndarray
    # If ‘coerce’, then invalid parsing will be set as NaN
    df = df[pd.to_numeric(df[column], errors='coerce').notnull()]

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [3]:
for i in range(0,len(df)):
    if not i==87 and not i==166 and not i==192 and not i==266 and not i==287:
        if df.loc[i, 'num'] >1:
            df.loc[i, 'num']=1
df.loc[299, 'num']=1
df.loc[300, 'num']=1

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,1
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,1
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [26]:
counter0=0
counter1=0
for i in range(0,len(df)):
    if not i==87 and not i==166 and not i==192 and not i==266 and not i==287:
        if df.loc[i, 'num'] == 1:
            counter1+=1
        elif df.loc[i, 'num'] == 0:
            counter0+=1

if df.loc[299, 'num'] == 1:
    counter1+=1
elif df.loc[299, 'num'] == 0:
    counter0+=1
    
if df.loc[300, 'num'] == 1:
    counter1+=1
elif df.loc[300, 'num'] == 0:
    counter0+=1

if df.loc[301, 'num'] == 1:
    counter1+=1
elif df.loc[301, 'num'] == 0:
    counter0+=1
print "counter0 = "+ str(counter0)
print "counter1 = "+ str(counter1)

counter0 = 160
counter1 = 135


In [4]:
print "min age = " + str(min(df.iloc[:,0]))
print "max age = "+ str(max(df.iloc[:,0]))
print "min restbps = " + str(min(df.iloc[:,3]))
print "max trestbps = " + str(max(df.iloc[:,3]))
print "min chol = " + str(min(df.iloc[:,4]))
print "max chol = "+ str(max(df.iloc[:,4]))
print "min thalach = " + str(min(df.iloc[:,7]))
print "max thalach = "+str(max(df.iloc[:,7]))
print "min oldpeak = " + str(min(df.iloc[:,9]))
print "max oldpeak = "+str(max(df.iloc[:,9]))

min age = 29.0
max age = 77.0
min restbps = 94.0
max trestbps = 200.0
min chol = 126.0
max chol = 564.0
min thalach = 71.0
max thalach = 202.0
min oldpeak = 0.0
max oldpeak = 6.2


# Conversion of categorical into bins

In [5]:
#age
out, bins =pd.qcut(df.iloc[:, 0], 4, labels=["Age: (29,48]", "Age: (48,56]","Age: (56,61]", "Age: (61,77]"], retbins= True)
df.iloc[:, 0] = out
#trestbps
out, bins = pd.qcut(df.iloc[:, 3], 4, labels=["trestbps: (94,120]", "trestbps: (120,130]","trestbps:(130,140]", "trestbsp: (140,200]"], retbins= True)
df.iloc[:, 3] = out
#chol
out, bins = pd.qcut(df.iloc[:, 4], 4, labels=["chol: (126,211]", "chol: (211,243]","chol: (243,276]", "chol: (276,564]"], retbins= True)
df.iloc[:, 4] = out
#thalach
out, bins = pd.qcut(df.iloc[:, 7], 4, labels=["thalach: (71,133]", "thalach: (133,153]","thalach: (153,166]", "thalach: (166,202]"], retbins= True)
df.iloc[:, 7] = out
#oldpeak
out, bins = pd.qcut(df.iloc[:, 9], 3, labels=["oldpeak: (0,0.1]", "oldpeak: (0.1,1.4]","oldpeak: (1.4,6.2]"], retbins= True)
df.iloc[:, 9] = out

In [6]:
#conversion check
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,"Age: (61,77]",1.0,1.0,"trestbsp: (140,200]","chol: (211,243]",1.0,2.0,"thalach: (133,153]",0.0,"oldpeak: (1.4,6.2]",3.0,0.0,6.0,0
1,"Age: (61,77]",1.0,4.0,"trestbsp: (140,200]","chol: (276,564]",0.0,2.0,"thalach: (71,133]",1.0,"oldpeak: (1.4,6.2]",2.0,3.0,3.0,1
2,"Age: (61,77]",1.0,4.0,"trestbps: (94,120]","chol: (211,243]",0.0,2.0,"thalach: (71,133]",1.0,"oldpeak: (1.4,6.2]",2.0,2.0,7.0,1
3,"Age: (29,48]",1.0,3.0,"trestbps: (120,130]","chol: (243,276]",0.0,0.0,"thalach: (166,202]",0.0,"oldpeak: (1.4,6.2]",3.0,0.0,3.0,0
4,"Age: (29,48]",0.0,2.0,"trestbps: (120,130]","chol: (126,211]",0.0,2.0,"thalach: (166,202]",0.0,"oldpeak: (0.1,1.4]",1.0,0.0,3.0,0
5,"Age: (48,56]",1.0,2.0,"trestbps: (94,120]","chol: (211,243]",0.0,0.0,"thalach: (166,202]",0.0,"oldpeak: (0.1,1.4]",1.0,0.0,3.0,0
6,"Age: (61,77]",0.0,4.0,"trestbps:(130,140]","chol: (243,276]",0.0,2.0,"thalach: (153,166]",0.0,"oldpeak: (1.4,6.2]",3.0,2.0,3.0,1
7,"Age:(56,61]",0.0,4.0,"trestbps: (94,120]","chol: (276,564]",0.0,0.0,"thalach: (153,166]",1.0,"oldpeak: (0.1,1.4]",1.0,0.0,3.0,0
8,"Age: (61,77]",1.0,4.0,"trestbps: (120,130]","chol: (243,276]",0.0,2.0,"thalach: (133,153]",0.0,"oldpeak: (0.1,1.4]",2.0,1.0,7.0,1
9,"Age: (48,56]",1.0,4.0,"trestbps:(130,140]","chol: (126,211]",1.0,2.0,"thalach: (153,166]",1.0,"oldpeak: (1.4,6.2]",3.0,0.0,7.0,1


# Bayesian network construction

In [83]:
#Model 1 is a 4 layer structure and the initial model
model1 = BayesianModel([('age', 'cp'),('age', 'trestbps'),('age', 'chol'), ('age', 'fbs'),('sex', 'cp') ,('sex', 'trestbps'), ('sex', 'chol'), ('sex','fbs'), ('cp', 'thalach'), ('cp', 'exang'), ('cp', 'oldpeak'), ('cp', 'slope'), ('cp', 'ca'),('cp', 'thal'),('cp', 'restecg'),('trestbps', 'restecg'),('trestbps', 'thalach'), ('trestbps','exang'), ('trestbps', 'oldpeak'), ('trestbps', 'slope'), ('trestbps', 'ca'), ('trestbps', 'thal'), ('chol', 'thalach'), ('chol', 'exang'), ('chol', 'oldpeak'), ('chol', 'slope'), ('chol', 'ca'), ('chol', 'thal'),('chol', 'restecg') ,('fbs', 'restecg'),('fbs', 'thalach'), ('fbs', 'exang'), ('fbs', 'oldpeak'), ('fbs', 'slope'), ('fbs', 'ca'), ('fbs', 'thal'),('thalach', 'num'), ('exang', 'num'), ('oldpeak', 'num'), ('slope', 'num'), ('ca', 'num'), ('thal', 'num'), ('restecg', 'num')])

model_from_hillclimbing = BayesianModel([('slope', 'oldpeak'), ('slope', 'thalach'), ('num', 'slope'), ('num', 'ca'), ('num', 'age'), ('num', 'thal'), ('cp', 'num'), ('cp', 'exang'), ('thal', 'sex')])

In [84]:
#estimation of parameters
model1.fit(df, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model1.get_cpds():
    print(cpd)

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
| age      | age(Age: (29,48]) | age(Age: (29,48]) | age(Age: (48,56]) | age(Age: (48,56]) | age(Age: (61,77]) | age(Age: (61,77]) | age(Age:(56,61]) | age(Age:(56,61]) |
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
| sex      | sex(0.0)          | sex(1.0)          | sex(0.0)          | sex(1.0)          | sex(0.0)          | sex(1.0)          | sex(0.0)         | sex(1.0)         |
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
| fbs(0.0) | 0.941988950276    | 0.926439232409    | 0.865482233503    | 0.789044289044    | 0.867816091954    | 0.790220820189    | 0.7684563758

+---------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+-----------------------------+-------

In [86]:
#estimation of parameters
model_from_hillclimbing.fit(df, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model_from_hillclimbing.get_cpds():
    print(cpd)

+------------+-----------------+-----------------+
| num        | num(0)          | num(1)          |
+------------+-----------------+-----------------+
| slope(1.0) | 0.638974358974  | 0.26403823178   |
+------------+-----------------+-----------------+
| slope(2.0) | 0.300512820513  | 0.643966547192  |
+------------+-----------------+-----------------+
| slope(3.0) | 0.0605128205128 | 0.0919952210275 |
+------------+-----------------+-----------------+
+------------+----------------+----------------+----------------+----------------+
| cp         | cp(1.0)        | cp(2.0)        | cp(3.0)        | cp(4.0)        |
+------------+----------------+----------------+----------------+----------------+
| exang(0.0) | 0.809278350515 | 0.907960199005 | 0.862017804154 | 0.451134380454 |
+------------+----------------+----------------+----------------+----------------+
| exang(1.0) | 0.190721649485 | 0.092039800995 | 0.137982195846 | 0.548865619546 |
+------------+----------------+------------

In [87]:
model1.get_cpds()

[<TabularCPD representing P(fbs:2 | age:4, sex:2) at 0x111ea8b50>,
 <TabularCPD representing P(slope:3 | chol:4, cp:4, fbs:2, trestbps:4) at 0x110971390>,
 <TabularCPD representing P(trestbps:4 | age:4, sex:2) at 0x110971110>,
 <TabularCPD representing P(exang:2 | chol:4, cp:4, fbs:2, trestbps:4) at 0x110971cd0>,
 <TabularCPD representing P(restecg:3 | chol:4, cp:4, fbs:2, trestbps:4) at 0x1109718d0>,
 <TabularCPD representing P(age:4) at 0x110971190>,
 <TabularCPD representing P(chol:4 | age:4, sex:2) at 0x110971fd0>,
 <TabularCPD representing P(sex:2) at 0x1107c5fd0>,
 <TabularCPD representing P(oldpeak:3 | chol:4, cp:4, fbs:2, trestbps:4) at 0x1112f3410>,
 <TabularCPD representing P(num:2 | ca:4, exang:2, oldpeak:3, restecg:3, slope:3, thal:3, thalach:4) at 0x10530a290>,
 <TabularCPD representing P(thalach:4 | chol:4, cp:4, fbs:2, trestbps:4) at 0x1107c5cd0>,
 <TabularCPD representing P(cp:4 | age:4, sex:2) at 0x1107c5d90>,
 <TabularCPD representing P(ca:4 | chol:4, cp:4, fbs:2, tre

In [88]:
model_from_hillclimbing.get_cpds()

[<TabularCPD representing P(slope:3 | num:2) at 0x1108fe4d0>,
 <TabularCPD representing P(exang:2 | cp:4) at 0x1108fe410>,
 <TabularCPD representing P(ca:4 | num:2) at 0x1108feb10>,
 <TabularCPD representing P(sex:2 | thal:3) at 0x1108feb50>,
 <TabularCPD representing P(oldpeak:3 | slope:3) at 0x1108c4410>,
 <TabularCPD representing P(num:2 | cp:4) at 0x1108fec10>,
 <TabularCPD representing P(thalach:4 | slope:3) at 0x1108fe910>,
 <TabularCPD representing P(cp:4) at 0x1108c4fd0>,
 <TabularCPD representing P(age:4 | num:2) at 0x1108c4d50>,
 <TabularCPD representing P(thal:3 | num:2) at 0x1108c4a10>]

In [37]:
#possible check for correctness cardinality of variables
model1.get_cardinality('num')

2

In [10]:
model1.check_model()

True

In [37]:
model_from_hillclimbing.check_model()

True

In [24]:
model1.local_independencies(['sex', 'age', 'fbs', 'slope', 'trestbps', 'exang', 'thalach', 'chol', 'oldpeak', 'restecg', 'cp', 'ca', 'thal', 'num'])

(sex _|_ age)
(age _|_ sex)
(fbs _|_ trestbps, cp, chol | age, sex)
(slope _|_ exang, restecg, age, sex, oldpeak, thalach, ca, thal | fbs, cp, trestbps, chol)
(trestbps _|_ fbs, cp, chol | age, sex)
(exang _|_ slope, restecg, age, sex, oldpeak, thalach, ca, thal | fbs, cp, trestbps, chol)
(thalach _|_ slope, exang, age, sex, oldpeak, restecg, ca, thal | fbs, cp, trestbps, chol)
(chol _|_ fbs, cp, trestbps | age, sex)
(oldpeak _|_ slope, exang, restecg, age, sex, thalach, ca, thal | fbs, cp, trestbps, chol)
(restecg _|_ slope, exang, age, sex, oldpeak, thalach, ca, thal | fbs, cp, trestbps, chol)
(cp _|_ fbs, trestbps, chol | age, sex)
(ca _|_ slope, exang, thalach, age, sex, oldpeak, restecg, thal | fbs, cp, trestbps, chol)
(thal _|_ slope, exang, thalach, age, sex, oldpeak, restecg, ca | fbs, cp, trestbps, chol)
(num _|_ fbs, trestbps, age, cp, chol, sex | slope, oldpeak, exang, thalach, restecg, ca, thal)

# Verification

### Scoring functions

In [91]:
# The overall BIC score of a model measures in how far the given model structure fits the data
# A negative value which is closer to 0 indicates a better fit
# See: Model Selection with BIC (http://www.cs.toronto.edu/~mbrubake/teaching/C11/Handouts/BIC.pdf)
bic = BicScore(df)
# print(bic.score(model2)) this one takes forever to run
print(bic.score(model1))
print(bic.score(model_from_hillclimbing))

KeyboardInterrupt: 

In [110]:
print(bic.local_score('age', parents=['sex', 'ca', 'oldpeak']))

# Determine which variables are good direct predictors based on their BIC score
print("age -> num: " + str(bic.local_score('num', parents=['age'])))
print("sex -> num: " + str(bic.local_score('num', parents=['sex'])))
print("cp -> num: " + str(bic.local_score('num', parents=['cp'])))
print("trestbps -> num: " + str(bic.local_score('num', parents=['trestbps'])))
print("chol -> num: " + str(bic.local_score('num', parents=['chol'])))
print("fbs -> num: " + str(bic.local_score('num', parents=['fbs'])))
print("restecg -> num: " + str(bic.local_score('num', parents=['restecg'])))
print("thalach -> num: " + str(bic.local_score('num', parents=['thalach'])))
print("exang -> num: " + str(bic.local_score('num', parents=['exang'])))
print("oldpeak -> num: " + str(bic.local_score('num', parents=['oldpeak'])))
print("slope -> num: " + str(bic.local_score('num', parents=['slope'])))
print("ca -> num: " + str(bic.local_score('num', parents=['ca'])))
print("thal -> num: " + str(bic.local_score('num', parents=['thal'])))

# Determine which BIC scores between age|sex and other variables
print("age -> sex: " + str(bic.local_score('sex', parents=['age'])))
print("age -> cp: " + str(bic.local_score('cp', parents=['age'])))
print("age -> trestbps: " + str(bic.local_score('trestbps', parents=['age'])))
print("age -> chol: " + str(bic.local_score('chol', parents=['age'])))
print("age -> fbs: " + str(bic.local_score('fbs', parents=['age'])))
print("age -> restecg: " + str(bic.local_score('restecg', parents=['age'])))
print("age -> thalach: " + str(bic.local_score('thalach', parents=['age'])))
print("age -> exang: " + str(bic.local_score('exang', parents=['age'])))
print("age -> oldpeak: " + str(bic.local_score('oldpeak', parents=['age'])))
print("age -> slope: " + str(bic.local_score('slope', parents=['age'])))
print("age -> ca: " + str(bic.local_score('ca', parents=['age'])))
print("age -> thal: " + str(bic.local_score('thal', parents=['age'])))

# Taking the two strongest predictors
print("thal, cp -> num: " + str(bic.local_score('num', parents=['thal', 'cp'])))

-552.391167347
age -> num: -203.202992799
sex -> num: -198.752767657
cp -> num: -175.763394346
trestbps -> num: -212.836333081
chol -> num: -213.289527994
fbs -> num: -210.665491289
restecg -> num: -208.681435692
thalach -> num: -190.432658998
exang -> num: -183.432111899
oldpeak -> num: -193.059271306
slope -> num: -191.120878949
ca -> num: -178.350813041
thal -> num: -170.234184358
age -> sex: -194.686588689
age -> cp: -382.904665848
age -> trestbps: -422.98223835
age -> chol: -440.152409668
age -> fbs: -130.736753115
age -> restecg: -243.283894642
age -> thalach: -416.791202619
age -> exang: -196.257355809
age -> oldpeak: -337.012844322
age -> slope: -284.58168973
age -> ca: -329.907094321
age -> thal: -271.821846715
thal, cp -> num: -166.38427889


In [61]:
# The following code would give you all possible independecies that someone could test
# model_from_hillclimbing.get_independencies()

In [62]:
# Performing HillClimbSearch to get a general idea of the possible structure
from pgmpy.estimators import HillClimbSearch

hc = HillClimbSearch(df, scoring_method=BicScore(df))
best_model = hc.estimate()
print(best_model.edges())

[('slope', 'oldpeak'), ('slope', 'thalach'), ('num', 'slope'), ('num', 'ca'), ('num', 'age'), ('num', 'thal'), ('cp', 'num'), ('cp', 'exang'), ('thal', 'sex')]


### (Conditional) Independence Tests

In [113]:
def is_independent(X, Y, Zs=[], significance_level=0.05):
    return est.test_conditional_independence(X, Y, Zs)[1] >= significance_level

est = ConstraintBasedEstimator(df)

print(est.test_conditional_independence('age', 'num'))
print(is_independent('age', 'num'))
print(est.test_conditional_independence('age', 'num', ['thalach']))
print(is_independent('age', 'num', ['thalach']))

# Checking independeces for a three layer architecture (model2)
# First layer: age, sex
# Second layer: Everything except age, sex, num
# Third layer : num
# age is indepent of num given variable_to_check

'''
print("Checking: 'age', 'sex'")
print(est.test_conditional_independence('age', 'sex'))
print(is_independent('age', 'sex'))

print("Checking: 'age', 'num', ['cp']")
print(est.test_conditional_independence('age', 'num', ['cp']))
print(is_independent('age', 'num', ['cp']))

print("Checking: 'age', 'num', ['trestbps']")
print(est.test_conditional_independence('age', 'num', ['trestbps']))
print(is_independent('age', 'num', ['trestbps']))

print("Checking: 'age', 'num', ['chol']")
print(est.test_conditional_independence('age', 'num', ['chol']))
print(is_independent('age', 'num', ['chol']))

print("Checking: 'age', 'num', ['fbs']")
print(est.test_conditional_independence('age', 'num', ['fbs']))
print(is_independent('age', 'num', ['fbs']))

print("Checking: 'age', 'num', ['restecg']")
print(est.test_conditional_independence('age', 'num', ['restecg']))
print(is_independent('age', 'num', ['restecg']))

print("Checking: 'age', 'num', ['thalach']")
print(est.test_conditional_independence('age', 'num', ['thalach']))
print(is_independent('age', 'num', ['thalach']))

print("Checking: 'age', 'num', ['exang']")
print(est.test_conditional_independence('age', 'num', ['exang']))
print(is_independent('age', 'num', ['exang']))

print("Checking: 'age', 'num', ['oldpeak']")
print(est.test_conditional_independence('age', 'num', ['oldpeak']))
print(is_independent('age', 'num', ['oldpeak']))

print("Checking: 'age', 'num', ['slope']")
print(est.test_conditional_independence('age', 'num', ['slope']))
print(is_independent('age', 'num', ['slope']))

print("Checking: 'age', 'num', ['ca']")
print(est.test_conditional_independence('age', 'num', ['ca']))
print(is_independent('age', 'num', ['ca']))

print("Checking: 'age', 'num', ['thal']")
print(est.test_conditional_independence('age', 'num', ['thal']))
print(is_independent('age', 'num', ['thal']))


# sex is indepent of num given variable_to_check
print("Checking: 'sex', 'num', ['cp']")
print(est.test_conditional_independence('sex', 'num', ['cp']))
print(is_independent('sex', 'num', ['cp']))

print("Checking: 'sex', 'num', ['trestbps']")
print(est.test_conditional_independence('sex', 'num', ['trestbps']))
print(is_independent('sex', 'num', ['trestbps']))

print("Checking: 'sex', 'num', ['chol']")
print(est.test_conditional_independence('sex', 'num', ['chol']))
print(is_independent('sex', 'num', ['chol']))

print("Checking: 'sex', 'num', ['fbs']")
print(est.test_conditional_independence('sex', 'num', ['fbs']))
print(is_independent('sex', 'num', ['fbs']))

print("Checking: 'sex', 'num', ['restecg']")
print(est.test_conditional_independence('sex', 'num', ['restecg']))
print(is_independent('sex', 'num', ['restecg']))

print("Checking: 'sex', 'num', ['thalach']")
print(est.test_conditional_independence('sex', 'num', ['thalach']))
print(is_independent('sex', 'num', ['thalach']))

print("Checking: 'sex', 'num', ['exang']")
print(est.test_conditional_independence('sex', 'num', ['exang']))
print(is_independent('sex', 'num', ['exang']))

print("Checking: 'sex', 'num', ['oldpeak']")
print(est.test_conditional_independence('sex', 'num', ['oldpeak']))
print(is_independent('sex', 'num', ['oldpeak']))

print("Checking: 'sex', 'num', ['slope']")
print(est.test_conditional_independence('sex', 'num', ['slope']))
print(is_independent('sex', 'num', ['slope']))

print("Checking: 'sex', 'num', ['ca']")
print(est.test_conditional_independence('sex', 'num', ['ca']))
print(is_independent('sex', 'num', ['ca']))

print("Checking: 'sex', 'num', ['thal']")
print(est.test_conditional_independence('sex', 'num', ['thal']))
print(is_independent('sex', 'num', ['thal']))

# Checking second layer to third layer

print("Checking: 'sex', 'num'")
print(est.test_conditional_independence('sex', 'num', ['thal']))
print(is_independent('sex', 'num', ['thal']))


#print(est.estimate(significance_level=0.05).edges())
'''

# Checking independeces for a four layer architecture (model1)
# First layer: age, sex
# Second layer: cp, trestbps, chol, fbs
# Third layer: restecg, thalach, exang, oldpeak, slope, ca, thal
# Fourth layer : num
#
# HillClimbing gave us the following connections which seem promising:
# [('slope', 'oldpeak'), ('slope', 'thalach'), ('num', 'slope'), 
# ('num', 'ca'), ('num', 'age'), ('num', 'thal'), ('cp', 'num'), 
# ('cp', 'exang'), ('thal', 'sex')]

print("*** Checking independeces for our four layer architecture ***")


# age ⊥ ca | chol, cp, fbs, trestbps
# leads to Insufficient data for testing age _|_ ca | ['chol', 'cp', 'fbs', 'trestbps']. 
# At least 5760 samples recommended, 297 present.
#print("Checking: 'age', 'ca', ['chol', 'cp', 'fbs', 'trestbps']")
#print(est.test_conditional_independence('age', 'ca', ['chol', 'cp', 'fbs', 'trestbps']))
#print(is_independent('age', 'ca', ['chol', 'cp', 'fbs', 'trestbps']))


# Checking age ⊥ sex
print("Checking: 'age', 'sex'")
print(est.test_conditional_independence('age', 'sex'))
print(is_independent('age', 'sex'))

# Checking chol ⊥ cp | age, sex
#print("Checking: 'chol', 'cp', ['age', 'sex']")
#print(est.test_conditional_independence('chol', 'cp', ['age', 'sex']))
#print(is_independent('chol', 'cp', ['age', 'sex']))
# /Users/christophschmidl/anaconda/lib/python2.7/site-packages/pgmpy/estimators/base.py:202: UserWarning:
# Insufficient data for testing chol _|_ cp | ['age', 'sex']. At least 360 samples recommended, 297 present.
# "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))

# Checking connection between first and second layer

# Checking chol ⊥ cp | age
print("Checking: 'chol', 'cp', ['age']")
print(est.test_conditional_independence('chol', 'cp', ['age']))
print(is_independent('chol', 'cp', ['age']))

# Checking chol ⊥ cp | sex
print("Checking: 'chol', 'cp', ['sex']")
print(est.test_conditional_independence('chol', 'cp', ['sex']))
print(is_independent('chol', 'cp', ['sex']))

# Checking chol ⊥ fbs | age
print("Checking: 'chol', 'fbs', ['age']")
print(est.test_conditional_independence('chol', 'fbs', ['age']))
print(is_independent('chol', 'fbs', ['age']))

# Checking chol ⊥ fbs | sex
print("Checking: 'chol', 'fbs', ['sex']")
print(est.test_conditional_independence('chol', 'fbs', ['sex']))
print(is_independent('chol', 'fbs', ['sex']))

# Checking chol ⊥ trestbps | age
print("Checking: 'chol', 'trestbps', ['age']")
print(est.test_conditional_independence('chol', 'trestbps', ['age']))
print(is_independent('chol', 'trestbps', ['age']))

# Checking chol ⊥ trestbps | sex
print("Checking: 'chol', 'trestbps', ['sex']")
print(est.test_conditional_independence('chol', 'trestbps', ['sex']))
print(is_independent('chol', 'trestbps', ['sex']))

# Checking cp ⊥ trestbps | age
print("Checking: 'cp', 'trestbps', ['age']")
print(est.test_conditional_independence('cp', 'trestbps', ['age']))
print(is_independent('cp', 'trestbps', ['age']))

# Checking cp ⊥ trestbps | sex
print("Checking: 'cp', 'trestbps', ['sex']")
print(est.test_conditional_independence('cp', 'trestbps', ['sex']))
print(is_independent('cp', 'trestbps', ['sex']))

# Checking cp ⊥ fbs | age
print("Checking: 'cp', 'fbs', ['age']")
print(est.test_conditional_independence('cp', 'fbs', ['age']))
print(is_independent('cp', 'fbs', ['age']))

# Checking cp ⊥ fbs | sex
print("Checking: 'cp', 'fbs', ['sex']")
print(est.test_conditional_independence('cp', 'fbs', ['sex']))
print(is_independent('cp', 'fbs', ['sex']))


# Checking fbs ⊥ trestbps | age
print("Checking: 'fbs', 'trestbps', ['age']")
print(est.test_conditional_independence('fbs', 'trestbps', ['age']))
print(is_independent('fbs', 'trestbps', ['age']))

# Checking fbs ⊥ trestbps | sex
print("Checking: 'fbs', 'trestbps', ['sex']")
print(est.test_conditional_independence('fbs', 'trestbps', ['sex']))
print(is_independent('fbs', 'trestbps', ['sex']))

# Checking last layer

print("Checking: 'age', 'num'")
print(est.test_conditional_independence('age', 'num'))
print(is_independent('age', 'num'))

print("Checking: 'sex', 'num'")
print(est.test_conditional_independence('sex', 'num'))
print(is_independent('sex', 'num'))

print("Checking: 'cp', 'num'")
print(est.test_conditional_independence('cp', 'num'))
print(is_independent('cp', 'num'))

print("Checking: 'trestbps', 'num'")
print(est.test_conditional_independence('trestbps', 'num'))
print(is_independent('trestbps', 'num'))

print("Checking: 'chol', 'num'")
print(est.test_conditional_independence('chol', 'num'))
print(is_independent('chol', 'num'))

print("Checking: 'fbs', 'num'")
print(est.test_conditional_independence('fbs', 'num'))
print(is_independent('fbs', 'num'))

print("Checking: 'restecg', 'num'")
print(est.test_conditional_independence('restecg', 'num'))
print(is_independent('restecg', 'num'))

print("Checking: 'thalach', 'num'")
print(est.test_conditional_independence('thalach', 'num'))
print(is_independent('thalach', 'num'))

print("Checking: 'exang', 'num'")
print(est.test_conditional_independence('exang', 'num'))
print(is_independent('exang', 'num'))

print("Checking: 'oldpeak', 'num'")
print(est.test_conditional_independence('oldpeak', 'num'))
print(is_independent('oldpeak', 'num'))

print("Checking: 'slope', 'num'")
print(est.test_conditional_independence('slope', 'num'))
print(is_independent('slope', 'num'))

print("Checking: 'ca', 'num'")
print(est.test_conditional_independence('ca', 'num'))
print(is_independent('ca', 'num'))

print("Checking: 'thal', 'num'")
print(est.test_conditional_independence('thal', 'num'))
print(is_independent('thal', 'num'))

# Checking age 
print("Checking: 'age', 'restecg'")
print(est.test_conditional_independence('age', 'restecg'))
print(is_independent('age', 'restecg'))

print("Checking: 'age', 'fbs'")
print(est.test_conditional_independence('age', 'fbs'))
print(is_independent('age', 'fbs'))

print("Checking: 'age', 'chol'")
print(est.test_conditional_independence('age', 'chol'))
print(is_independent('age', 'chol'))

print("Checking: 'age', 'trestbps'")
print(est.test_conditional_independence('age', 'trestbps'))
print(is_independent('age', 'trestbps'))

# Checking sex
print("Checking: 'sex', 'restecg'")
print(est.test_conditional_independence('sex', 'restecg'))
print(is_independent('sex', 'restecg'))

print("Checking: 'sex', 'fbs'")
print(est.test_conditional_independence('sex', 'fbs'))
print(is_independent('sex', 'fbs'))

print("Checking: 'sex', 'chol'")
print(est.test_conditional_independence('sex', 'chol'))
print(is_independent('age', 'chol'))

print("Checking: 'sex', 'trestbps'")
print(est.test_conditional_independence('sex', 'trestbps'))
print(is_independent('sex', 'trestbps'))




(25.761327333596547, 0.00055558553299022773, True)
False
(34.537538087345197, 0.30248920208931868, True)
True
*** Checking independeces for our four layer architecture ***
Checking: 'age', 'sex'
(7.3983096971674334, 0.3886178641152912, True)
True
Checking: 'chol', 'cp', ['age']
(48.755980318815176, 0.90641007049859712, True)
True
Checking: 'chol', 'cp', ['sex']
(19.356336027916605, 0.94861855064178746, True)
True
Checking: 'chol', 'fbs', ['age']
(8.5220117545518352, 0.99997916899152528, True)
True
Checking: 'chol', 'fbs', ['sex']
(7.7317645898847749, 0.93408570959399273, True)
True
Checking: 'chol', 'trestbps', ['age']
(52.294371354181479, 0.82983578179065598, True)
True
Checking: 'chol', 'trestbps', ['sex']
(29.535048083335614, 0.54138882596987115, True)
True
Checking: 'cp', 'trestbps', ['age']
(35.60149137611478, 0.99790544700736195, True)
True
Checking: 'cp', 'trestbps', ['sex']
(32.797499686243945, 0.37888554292260285, True)
True
Checking: 'cp', 'fbs', ['age']
(13.876112017395243, 

# Inference on first model

for ca the order for likelihood of heartdisease is reversed: 0 (worst) - 3(best).
for slope flat (1) is the worst. 
CP value 4 seems to be really predictive


In [ ]:
predict num
inference on sex =1 and age bin 3 (combinations of male and female and ages)
inference on slope and ca. (all combinations)
inference on cp (all combinations)

In [12]:
infer1 = VariableElimination(model1)

In [22]:
print "num = \n" + str(infer1.query(['num']) ['num'])

num = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5624 |
| num_1 |     0.4376 |
+-------+------------+


In [23]:
#predicition of num given sex and ages
print "sex:0 , age:0 = \n" + str(infer1.query(['num'], evidence={'sex': 0,'age': 0}) ['num'])
print "sex:0 , age:1 = \n" + str(infer1.query(['num'], evidence={'sex': 0,'age': 1}) ['num'])
print "sex:0 , age:2 = \n" + str(infer1.query(['num'], evidence={'sex': 0,'age': 2}) ['num'])
print "sex:0 , age:3 = \n" + str(infer1.query(['num'], evidence={'sex': 0,'age': 3}) ['num'])
print "sex:1 , age:0 = \n" + str(infer1.query(['num'], evidence={'sex': 1,'age': 0}) ['num'])
print "sex:1 , age:1 = \n" + str(infer1.query(['num'], evidence={'sex': 1,'age': 1}) ['num'])
print "sex:1 , age:2 = \n" + str(infer1.query(['num'], evidence={'sex': 1,'age': 2}) ['num'])
print "sex:1 , age:3 = \n" + str(infer1.query(['num'], evidence={'sex': 1,'age': 3}) ['num'])

sex:0 , age:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.6661 |
| num_1 |     0.3339 |
+-------+------------+
sex:0 , age:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.6008 |
| num_1 |     0.3992 |
+-------+------------+
sex:0 , age:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5446 |
| num_1 |     0.4554 |
+-------+------------+
sex:0 , age:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5102 |
| num_1 |     0.4898 |
+-------+------------+
sex:1 , age:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5902 |
| num_1 |     0.4098 |
+-------+------------+
sex:1 , age:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5682 |
| num_1 |     0.4318 |
+-------+------------+
sex:1 , age:2 = 
+-------+------------+
| num   |   phi(num) |
|------

In [20]:
#prediction of slope and ca
print "slope:0 , ca:0 = \n" + str(infer1.query(['num'], evidence={'slope': 0, 'ca': 0}) ['num'])
print "slope:0 , ca:1 = \n" + str(infer1.query(['num'], evidence={'slope': 0, 'ca': 1}) ['num'])
print "slope:0 , ca:2 = \n" + str(infer1.query(['num'], evidence={'slope': 0, 'ca': 2}) ['num'])
print "slope:0 , ca:3 = \n" + str(infer1.query(['num'], evidence={'slope': 0, 'ca': 3}) ['num'])
print "slope:1 , ca:0 = \n" + str(infer1.query(['num'], evidence={'slope': 1, 'ca': 0}) ['num'])
print "slope:1 , ca:1 = \n" + str(infer1.query(['num'], evidence={'slope': 1, 'ca': 1}) ['num'])
print "slope:1 , ca:2 = \n" + str(infer1.query(['num'], evidence={'slope': 1, 'ca': 2}) ['num'])
print "slope:1 , ca:3 = \n" + str(infer1.query(['num'], evidence={'slope': 1, 'ca': 3}) ['num'])
print "slope:2 , ca:0 = \n" + str(infer1.query(['num'], evidence={'slope': 2, 'ca': 0}) ['num'])
print "slope:2 , ca:1 = \n" + str(infer1.query(['num'], evidence={'slope': 2, 'ca': 1}) ['num'])
print "slope:2 , ca:2 = \n" + str(infer1.query(['num'], evidence={'slope': 2, 'ca': 2}) ['num'])
print "slope:2 , ca:3 = \n" + str(infer1.query(['num'], evidence={'slope': 2, 'ca': 3}) ['num'])

slope:0 , ca:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.7405 |
| num_1 |     0.2595 |
+-------+------------+
slope:0 , ca:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5508 |
| num_1 |     0.4492 |
+-------+------------+
slope:0 , ca:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4928 |
| num_1 |     0.5072 |
+-------+------------+
slope:0 , ca:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4900 |
| num_1 |     0.5100 |
+-------+------------+
slope:1 , ca:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5528 |
| num_1 |     0.4472 |
+-------+------------+
slope:1 , ca:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.3931 |
| num_1 |     0.6069 |
+-------+------------+
slope:1 , ca:2 = 
+-------+------------+
| num   |   phi(num) |


In [21]:
#inference on cp (all combinations)
print "cp:0 = \n" + str(infer1.query(['num'], evidence={'cp': 0}) ['num'])
print "cp:1 = \n" + str(infer1.query(['num'], evidence={'cp': 1}) ['num'])
print "cp:2 = \n" + str(infer1.query(['num'], evidence={'cp': 2}) ['num'])
print "cp:3 = \n" + str(infer1.query(['num'], evidence={'cp': 3}) ['num'])

cp:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4833 |
| num_1 |     0.5167 |
+-------+------------+
cp:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.7370 |
| num_1 |     0.2630 |
+-------+------------+
cp:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.6391 |
| num_1 |     0.3609 |
+-------+------------+
cp:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4693 |
| num_1 |     0.5307 |
+-------+------------+


# Final Model

In [7]:
final_model = BayesianModel([('sex', 'num'),('age', 'trestbps'), ('age','num'), ('exang','cp'), ('exang', 'num'), ('cp','num'), ('thal','num'), ('ca', 'num'), ('slope','num'),('slope','oldpeak'),('slope','thalach'),('oldpeak','num'),('thalach', 'num')])

In [16]:
#estimation of parameters
final_model.fit(df, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in final_model.get_cpds():
    print(cpd)

+------------+-----------+
| slope(1.0) | 0.465784  |
+------------+-----------+
| slope(2.0) | 0.459161  |
+------------+-----------+
| slope(3.0) | 0.0750552 |
+------------+-----------+
+-------------------------------+-------------------+-------------------+-------------------+------------------+
| age                           | age(Age: (29,48]) | age(Age: (48,56]) | age(Age: (61,77]) | age(Age:(56,61]) |
+-------------------------------+-------------------+-------------------+-------------------+------------------+
| trestbps(trestbps: (120,130]) | 0.25              | 0.25              | 0.25              | 0.25             |
+-------------------------------+-------------------+-------------------+-------------------+------------------+
| trestbps(trestbps: (94,120])  | 0.25              | 0.25              | 0.25              | 0.25             |
+-------------------------------+-------------------+-------------------+-------------------+------------------+
| trestbps(trestbps:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
final_model.check_model()

True

In [10]:
final_model.local_independencies(['sex', 'age', 'slope', 'trestbps', 'exang', 'thalach', 'oldpeak', 'cp', 'ca', 'thal', 'num'])

(sex _|_ slope, trestbps, exang, age, oldpeak, num, thalach, cp, ca, thal)
(age _|_ slope, trestbps, exang, ca, sex, oldpeak, num, thalach, cp, thal)
(slope _|_ trestbps, exang, age, sex, oldpeak, num, thalach, cp, ca, thal)
(trestbps _|_ slope, exang, ca, sex, oldpeak, num, thalach, cp, thal | age)
(exang _|_ slope, trestbps, age, sex, oldpeak, num, thalach, cp, ca, thal)
(thalach _|_ trestbps, exang, age, sex, oldpeak, num, cp, ca, thal | slope)
(oldpeak _|_ trestbps, exang, age, sex, num, thalach, cp, ca, thal | slope)
(cp _|_ slope, trestbps, age, sex, oldpeak, num, thalach, ca, thal | exang)
(ca _|_ slope, trestbps, exang, age, sex, oldpeak, num, thalach, cp, thal)
(thal _|_ slope, trestbps, exang, age, sex, oldpeak, num, thalach, cp, ca)
(num _|_ trestbps | slope, exang, ca, sex, oldpeak, thalach, cp, age, thal)

## Inferences run on Final Model

In [11]:
infer_final = VariableElimination(final_model)

In [12]:
print "num = \n" + str(infer_final.query(['num']) ['num'])

num = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5063 |
| num_1 |     0.4937 |
+-------+------------+


In [13]:
#predicition of num given sex and ages
print "sex:0 , age:0 = \n" + str(infer_final.query(['num'], evidence={'sex': 0,'age': 0}) ['num'])
print "sex:0 , age:1 = \n" + str(infer_final.query(['num'], evidence={'sex': 0,'age': 1}) ['num'])
print "sex:0 , age:2 = \n" + str(infer_final.query(['num'], evidence={'sex': 0,'age': 2}) ['num'])
print "sex:0 , age:3 = \n" + str(infer_final.query(['num'], evidence={'sex': 0,'age': 3}) ['num'])
print "sex:1 , age:0 = \n" + str(infer_final.query(['num'], evidence={'sex': 1,'age': 0}) ['num'])
print "sex:1 , age:1 = \n" + str(infer_final.query(['num'], evidence={'sex': 1,'age': 1}) ['num'])
print "sex:1 , age:2 = \n" + str(infer_final.query(['num'], evidence={'sex': 1,'age': 2}) ['num'])
print "sex:1 , age:3 = \n" + str(infer_final.query(['num'], evidence={'sex': 1,'age': 3}) ['num'])

sex:0 , age:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5154 |
| num_1 |     0.4846 |
+-------+------------+
sex:0 , age:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5112 |
| num_1 |     0.4888 |
+-------+------------+
sex:0 , age:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5104 |
| num_1 |     0.4896 |
+-------+------------+
sex:0 , age:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5024 |
| num_1 |     0.4976 |
+-------+------------+
sex:1 , age:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5094 |
| num_1 |     0.4906 |
+-------+------------+
sex:1 , age:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5101 |
| num_1 |     0.4899 |
+-------+------------+
sex:1 , age:2 = 
+-------+------------+
| num   |   phi(num) |
|------

In [17]:
#prediction of slope and ca
print "slope:0 , ca:0 = \n" + str(infer_final.query(['num'], evidence={'slope': 0, 'ca': 0}) ['num'])
print "slope:0 , ca:1 = \n" + str(infer_final.query(['num'], evidence={'slope': 0, 'ca': 1}) ['num'])
print "slope:0 , ca:2 = \n" + str(infer_final.query(['num'], evidence={'slope': 0, 'ca': 2}) ['num'])
print "slope:0 , ca:3 = \n" + str(infer_final.query(['num'], evidence={'slope': 0, 'ca': 3}) ['num'])
print "slope:1 , ca:0 = \n" + str(infer_final.query(['num'], evidence={'slope': 1, 'ca': 0}) ['num'])
print "slope:1 , ca:1 = \n" + str(infer_final.query(['num'], evidence={'slope': 1, 'ca': 1}) ['num'])
print "slope:1 , ca:2 = \n" + str(infer_final.query(['num'], evidence={'slope': 1, 'ca': 2}) ['num'])
print "slope:1 , ca:3 = \n" + str(infer_final.query(['num'], evidence={'slope': 1, 'ca': 3}) ['num'])
print "slope:2 , ca:0 = \n" + str(infer_final.query(['num'], evidence={'slope': 2, 'ca': 0}) ['num'])
print "slope:2 , ca:1 = \n" + str(infer_final.query(['num'], evidence={'slope': 2, 'ca': 1}) ['num'])
print "slope:2 , ca:2 = \n" + str(infer_final.query(['num'], evidence={'slope': 2, 'ca': 2}) ['num'])
print "slope:2 , ca:3 = \n" + str(infer_final.query(['num'], evidence={'slope': 2, 'ca': 3}) ['num'])

slope:0 , ca:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5203 |
| num_1 |     0.4797 |
+-------+------------+
slope:0 , ca:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5008 |
| num_1 |     0.4992 |
+-------+------------+
slope:0 , ca:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4996 |
| num_1 |     0.5004 |
+-------+------------+
slope:0 , ca:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5000 |
| num_1 |     0.5000 |
+-------+------------+
slope:1 , ca:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5062 |
| num_1 |     0.4938 |
+-------+------------+
slope:1 , ca:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4937 |
| num_1 |     0.5063 |
+-------+------------+
slope:1 , ca:2 = 
+-------+------------+
| num   |   phi(num) |


In [21]:
#inference on cp (all combinations)
print "cp:0 = \n" + str(infer_final.query(['num'], evidence={'cp': 0}) ['num'])
print "cp:1 = \n" + str(infer_final.query(['num'], evidence={'cp': 1}) ['num'])
print "cp:2 = \n" + str(infer_final.query(['num'], evidence={'cp': 2}) ['num'])
print "cp:3 = \n" + str(infer_final.query(['num'], evidence={'cp': 3}) ['num'])

cp:0 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5019 |
| num_1 |     0.4981 |
+-------+------------+
cp:1 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5098 |
| num_1 |     0.4902 |
+-------+------------+
cp:2 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5140 |
| num_1 |     0.4860 |
+-------+------------+
cp:3 = 
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4996 |
| num_1 |     0.5004 |
+-------+------------+


In [19]:
print(infer_final.query(['num'], evidence={'sex': 1, 'age': 3, 'cp': 3, 'trestbps': 0, 'ca': 3, 'exang': 1,'oldpeak': 2, 'slope': 1, 'thal':2 }) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.3750 |
| num_1 |     0.6250 |
+-------+------------+


# Additional tests ran

In [39]:
# Basic inference on num, without observed variables. Note: takes approx 15 min for me.
infer = VariableElimination(model)
print(infer.query(['num']) ['num'])

KeyboardInterrupt: 

In [40]:
infer = VariableElimination(model)
print(infer.query(['num']) ['num'])
print(infer.query(['num'], evidence={'sex': 0}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5397 |
| num_1 |     0.4603 |
+-------+------------+
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5702 |
| num_1 |     0.4298 |
+-------+------------+


In [123]:
infer1 = VariableElimination(model1)
print(infer1.query(['num']) ['num'])
print(infer1.query(['num'], evidence={'sex': 0}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5624 |
| num_1 |     0.4376 |
+-------+------------+
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5839 |
| num_1 |     0.4161 |
+-------+------------+


In [127]:
infer2 = VariableElimination(model2)
print(infer2.query(['num']) ['num'])
print(infer2.query(['num'], evidence={'sex': 0}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5048 |
| num_1 |     0.4952 |
+-------+------------+
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5090 |
| num_1 |     0.4910 |
+-------+------------+


In [128]:
print(infer2.query(['num'], evidence={'sex': 1, 'age': 1, 'cp': 3, 'chol': 3, 'trestbps': 3, 'fbs': 1, 'restecg': 2,'ca': 3, 'exang': 1,'oldpeak': 2, 'slope': 2, 'thal': 2 }) ['num'])
print(infer2.query(['num'], evidence={'sex': 1, 'age': 3, 'cp': 3, 'chol': 3, 'trestbps': 0, 'fbs': 0, 'restecg': 0,'ca': 3, 'exang': 1,'oldpeak': 2, 'slope': 1, 'thal':2 }) ['num'])
print(infer2.query(['num'], evidence={'sex': 0, 'age': 0, 'cp': 0, 'chol': 0, 'trestbps': 1, 'fbs': 1, 'restecg': 1,'ca': 3, 'exang': 1,'oldpeak': 2 }) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5000 |
| num_1 |     0.5000 |
+-------+------------+
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5000 |
| num_1 |     0.5000 |
+-------+------------+
+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5000 |
| num_1 |     0.5000 |
+-------+------------+


In [103]:
print(infer.query(['num'], evidence={'sex': 1}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5249 |
| num_1 |     0.4751 |
+-------+------------+


In [100]:
print(infer.query(['num'], evidence={'restecg': 0}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5537 |
| num_1 |     0.4463 |
+-------+------------+


In [101]:
print(infer.query(['num'], evidence={'restecg': 1}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4599 |
| num_1 |     0.5401 |
+-------+------------+


In [102]:
print(infer.query(['num'], evidence={'restecg': 2}) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5289 |
| num_1 |     0.4711 |
+-------+------------+


In [88]:
print(infer.query(['num'], evidence={'sex': 1, 'age': 1, 'cp': 3, 'chol': 3, 'trestbps': 3, 'fbs': 1, 'restecg': 2,'ca': 3, 'exang': 1,'oldpeak': 2, 'slope': 2, 'thal': 2 }) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.5000 |
| num_1 |     0.5000 |
+-------+------------+


In [96]:
print(infer.query(['num'], evidence={'sex': 1, 'age': 3, 'cp': 3, 'chol': 3, 'trestbps': 0, 'fbs': 0, 'restecg': 0,'ca': 3, 'exang': 1,'oldpeak': 2, 'slope': 1, 'thal':2 }) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.3754 |
| num_1 |     0.6246 |
+-------+------------+


In [59]:
print(infer.query(['num'], evidence={'sex': 0, 'age': 0, 'cp': 0, 'chol': 0, 'trestbps': 1, 'fbs': 1, 'restecg': 1,'ca': 3, 'exang': 1,'oldpeak': 2 }) ['num'])

+-------+------------+
| num   |   phi(num) |
|-------+------------|
| num_0 |     0.4585 |
| num_1 |     0.5415 |
+-------+------------+
